In [1]:
DEEPL_API_KEY = ""
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=iXGZDtc8dmk" # "https://youtu.be/B8ocYx5_64o?si=f4UIdzTXkGCIkv4X"
TARGET_LANGUAGE = "DE"

In [2]:
ffmpeg_path = '/usr/bin/ffmpeg'
video_codec = 'hevc_nvenc'
audio_codec = 'aac'
audio_bitrate = '128k'

# Importing dependencies

In [3]:
!sudo apt update -q && sudo apt install -y -q ffmpeg

Get:1 file:/var/cuda-repo-wsl-ubuntu-12-4-local  InRelease [1572 B]
Get:1 file:/var/cuda-repo-wsl-ubuntu-12-4-local  InRelease [1572 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Reading package lists...
Building dependency tree...
Reading state information...
39 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [4]:
%pip install --quiet pytube openai-whisper deepl git+https://github.com/openai/whisper.git TTS python-ffmpeg~=2.0 groq
![[ -e "cache" ]] || mkdir cache

Note: you may need to restart the kernel to use updated packages.


In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Work with Audio

In [6]:
from pytube import YouTube
yt = YouTube(YOUTUBE_VIDEO_URL)

In [7]:
stream = yt.streams.filter(only_audio=True).first()
dst_path = "cache"
out_file = stream.download(output_path=dst_path, filename="original_audio.mp4")

In [8]:
out_file

'/mnt/c/Users/Sergey/Documents/Projects/video_translator/cache/original_audio.mp4'

## Convert Audio to Text

In [9]:
import torch
# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")
if device == "cuda":
    torch.backends.cudnn.enabled = False

Running on cuda


Load whisper model

In [10]:
import whisper
model = whisper.load_model("large-v3", download_root="cache", device="cpu")

Transcribe audio:

In [11]:
%%time
result = model.transcribe(out_file, word_timestamps=True)

/home/sergey/miniconda3/envs/pytrans/lib/python3.11/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/sergey/miniconda3/envs/pytrans/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


CPU times: user 44min 51s, sys: 45.2 s, total: 45min 37s
Wall time: 6min 7s


In [12]:
result['text']

" Вот это рекорд. Короче, презентации Apple я, пожалуй, что и не вспомню. Отстрелялись за 35 минут. Ну а я могу еще быстрее. Цените, как сэкономлю вам время. Сначала был Тим Кук с его фирменным... Good morning. Welcome to Apple Park. Какой morning, Тим? Пять вечера по Москве, алё. Тем не менее, вспомнили Vision Pro и мартовский апдейт Air'ов. Дескать, их так любят все вокруг, что теперь это самый лучший ноут на 13 и 15 дюймов. И дальше сразу пошли iPad'ы. Сначала Air. Теперь он в двух размерах, на 11 и на 13 дюймов. У 13-дюймовой версии экран больше аж на 30%, а это значит, правильно, что больше людей влезет в групповой созвон по зуму. На месте осталось пространственное аудио, зато навалили басов изрядно. Цветов 4, все-таки приятненькие, но вы же все равно купите самый темный, ну. По железу. Внутри теперь M.2, который, очевидно, во всем быстрее. Ну и было бы странно, если бы он был медленнее, согласитесь. Особенно прирост в мощности заметен на фоне Air'а. На A14 тут аж в три раза быстр

## Segment into sentences with timestamps and translations

In [13]:
import re
def find_sentence_end(text):
    # Regular expression to match a sentence ending with a period, question mark, or exclamation mark
    pattern = r"[.!?](?=\s|$)"

    # Find all matches in the text
    match = re.search(pattern, text)
    if match:
        return match.end()
    else:
        return -1

segments = []
partial_text = ""
start_time = 0
for s in result['segments']:
    word_num = 0
    if start_time == -1:
        start_time = s['start']
    partial_text += s['text']
    sentence_end = find_sentence_end(partial_text)
    while sentence_end > 0:
        sentence = partial_text[:sentence_end].strip()
        last_word = sentence.split()[-1]
        next_start_time = s['end']
        current_end_time = s['end']
        while word_num < len(s["words"]):
            word_num += 1
            if s['words'][word_num - 1]['word'].strip() == last_word:
                current_end_time = s['words'][word_num - 1]['end']
                if word_num < len(s["words"]):
                    next_start_time = s["words"][word_num]["start"]
                else:
                    next_start_time = -1
                break
        if len(segments) > 0 and segments[-1]['end'] - start_time < 0.5 and segments[-1]['end'] - segments[-1]['start'] < 15:
            # Very small pause between sentence and previous segment is small enough. Append it to previous segment
            segments[-1]['text'] += f' {sentence}'
            segments[-1]['end'] = current_end_time
        else:
            segments.append({
                'text': sentence,
                'start': start_time,
                'end': current_end_time,
            })
        start_time = next_start_time
        partial_text = partial_text[sentence_end:]
        sentence_end = find_sentence_end(partial_text)

For debug purposes

In [14]:
for i in range(0, len(segments)):
    print(f"{i} {segments[i]['start']} - {segments[i]['end']}: {segments[i]['text']}")

0 0 - 15.9: Вот это рекорд. Короче, презентации Apple я, пожалуй, что и не вспомню. Отстрелялись за 35 минут. Ну а я могу еще быстрее. Цените, как сэкономлю вам время. Сначала был Тим Кук с его фирменным... Good morning. Welcome to Apple Park. Какой morning, Тим?
1 15.9 - 32.56: Пять вечера по Москве, алё. Тем не менее, вспомнили Vision Pro и мартовский апдейт Air'ов. Дескать, их так любят все вокруг, что теперь это самый лучший ноут на 13 и 15 дюймов. И дальше сразу пошли iPad'ы. Сначала Air. Теперь он в двух размерах, на 11 и на 13 дюймов.
2 32.68 - 49.28: У 13-дюймовой версии экран больше аж на 30%, а это значит, правильно, что больше людей влезет в групповой созвон по зуму. На месте осталось пространственное аудио, зато навалили басов изрядно. Цветов 4, все-таки приятненькие, но вы же все равно купите самый темный, ну.
3 49.36 - 64.62: По железу. Внутри теперь M.2, который, очевидно, во всем быстрее. Ну и было бы странно, если бы он был медленнее, согласитесь. Особенно прирост в мо

## Translate segments

In [15]:
%%time
import deepl
translator = deepl.Translator(DEEPL_API_KEY)

for s in segments:
    deepl_result = translator.translate_text(s['text'], target_lang=TARGET_LANGUAGE)
    s['translated'] = deepl_result.text

CPU times: user 57.9 ms, sys: 24 µs, total: 58 ms
Wall time: 5.16 s


Debug the translation

In [16]:
for i in range(0, len(segments)):
    print(f"{i} {segments[i]['start']} - {segments[i]['end']}: {segments[i]['translated']} | {segments[i]['text']}")

0 0 - 15.9: Das ist ein Rekord. Ich glaube nicht, dass ich mich jemals an eine Apple Präsentation erinnern werde. Wir haben sie in 35 Minuten aufgenommen. Nun, ich kann es sogar noch schneller machen. Schätzen Sie die Zeit, die ich Ihnen erspare. Den Anfang machte Tim Cook mit seinem Markenzeichen... Guten Morgen. Willkommen im Apple Park. Was für ein Morgen, Tim? | Вот это рекорд. Короче, презентации Apple я, пожалуй, что и не вспомню. Отстрелялись за 35 минут. Ну а я могу еще быстрее. Цените, как сэкономлю вам время. Сначала был Тим Кук с его фирменным... Good morning. Welcome to Apple Park. Какой morning, Тим?
1 15.9 - 32.56: 17 Uhr Moskauer Zeit, hallo. Nichtsdestotrotz wurden der Vision Pro und das März-Update des Air's in Erinnerung gerufen. Sie sagen, dass jeder sie so sehr liebt, dass es jetzt der beste Laptop bei 13 und 15 Zoll ist. Und dann kamen die iPads. Zuerst das Air. Jetzt gibt es sie in zwei Größen, mit 11 und 13 Zoll. | Пять вечера по Москве, алё. Тем не менее, вспомн

## Make translated audio

Make a sample audio to copy voice

In [17]:
from random import randint
from ffmpeg import FFmpeg

def make_sample_audio(start, end, src_file, out_file):
    # ffmpeg -i /content/1voAY7tAXAA.mp4 -acodec pcm_s16le -ar 22050 -ac 2 /content/sample.wav
    ffmpeg = (
        FFmpeg(ffmpeg_path)
        .option('y')
        .input(src_file)
        .output(
            out_file, 
            acodec="pcm_s16le", 
            ar=24000,
            ac=2,
            af=f"[0]atrim=start={start:.2f}:end={end:.2f}[t0]"
        )
    )
    print(ffmpeg.arguments)
    ffmpeg.execute()

Create TTS engine

In [32]:
%%time
from TTS.api import TTS

multi_language = True
multi_speaker = True
# Init TTS
if TARGET_LANGUAGE.lower() == 'de':
    tts = TTS("tts_models/de/thorsten/vits").to(device)
    # tts = TTS("tts_models/de/thorsten/tacotron2-DDC").to(device)
    multi_language = False
    multi_speaker = False
else:
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Only english, french and Portuguese
# tts = TTS("tts_models/multilingual/multi-dataset/your_tts").to("cuda")


 > tts_models/de/thorsten/vits is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
CPU times: user 2.57 s, sys: 50.1 ms, total: 2.62 s
Wall time: 874 ms


In [33]:
%%time
from ipywidgets import IntProgress
from IPython.display import display
from random import randint

f = IntProgress(min=0, max=len(segments)) # instantiate the bar
display(f)
sample_segment = randint(0, len(segments))
sample_wav = f'cache/{sample_segment}_sample.wav'
make_sample_audio(segments[sample_segment]['start'], segments[sample_segment]['end'], out_file, sample_wav)

for i in range(0, len(segments)):
    tts.tts_to_file(
        text=segments[i]['translated'],
        speaker_wav=sample_wav if multi_speaker else None,
        language=TARGET_LANGUAGE.lower() if multi_language else None,
        file_path=f"cache/{i}.wav"
    )
    f.value += 1

IntProgress(value=0, max=11)

['/usr/bin/ffmpeg', '-y', '-i', '/mnt/c/Users/Sergey/Documents/Projects/video_translator/cache/original_audio.mp4', '-acodec', 'pcm_s16le', '-ar', '24000', '-ac', '2', '-af', '[0]atrim=start=15.90:end=32.56[t0]', 'cache/1_sample.wav']
 > Text splitted to sentences.
['Das ist ein Rekord.', 'Ich glaube nicht, dass ich mich jemals an eine Apple Präsentation erinnern werde.', 'Wir haben sie in 35 Minuten aufgenommen.', 'Nun, ich kann es sogar noch schneller machen.', 'Schätzen Sie die Zeit, die ich Ihnen erspare.', 'Den Anfang machte Tim Cook mit seinem Markenzeichen...', 'Guten Morgen.', 'Willkommen im Apple Park.', 'Was für ein Morgen, Tim?']
ɪç ɡlaoːbə nɪçt, das ɪç mɪç jeːmals ʔan ʔaɪ̯nə ʔapplə pʁɛzɛntatsɪoːn ʔɛːɐʔɪnnɛʁn vɛʁdə.
 [!] Character '̯' not found in the vocabulary. Discarding it.
 > Processing time: 1.011486530303955
 > Real-time factor: 0.03728647517921996
 > Text splitted to sentences.
['17 Uhr Moskauer Zeit, hallo.', "Nichtsdestotrotz wurden der Vision Pro und das März-Upda

Now fix audio speed to match original audio track

In [34]:
import subprocess
import shlex
import shutil
import json

def get_audio_duration(path):
    cmd = "ffprobe -v quiet -print_format json -show_streams"
    args = shlex.split(cmd)
    args.append(path)
    # run the ffprobe process, decode stdout into utf-8 & convert to JSON
    ffprobeOutput = subprocess.check_output(args).decode("utf-8")
    ffprobeOutput = json.loads(ffprobeOutput)
    return float(ffprobeOutput["streams"][0]["duration"])

translated_total_duration = 0
original_audio_duration = get_audio_duration(out_file)

for i in range(0, len(segments)):
    translated_audio = f"cache/{i}.wav"
    tempo_audio = f"cache/{i}_tempo.wav"
    translated_duration = get_audio_duration(translated_audio)
    original_duration = segments[i]['end'] - segments[i]['start']
    
    if original_duration < translated_duration:
        # Original is shorter than translated audio. Increase sound tempo
        atempo = translated_duration / original_duration
        if atempo > 1.35:
            atempo = 1.35
        ffmpeg = (
            FFmpeg()
            .option("y")
            .input(translated_audio)
            .output(
                tempo_audio,
                acodec="pcm_s16le",
                ar=24000,
                ac=2,
                af=f"[0]atempo={atempo:.2f}[t0]",
            )
        )
        ffmpeg.execute()
        shutil.move(tempo_audio, translated_audio)
    translated_total_duration += get_audio_duration(translated_audio)

# Work with video

Download video file

In [35]:
stream = yt.streams.filter(resolution="720p").first()
vid_file = stream.download(output_path=dst_path, filename="video.mp4")

Combine audio segments with video

In [36]:
%%time
import os

for i in range(0, len(segments)):
    audio_duration = get_audio_duration(f'cache/{i}.wav')
    video_duration = segments[i]['end'] - segments[i]['start']
    pts_ratio = audio_duration / video_duration
    if pts_ratio < 1.0:
        pts_ration = 1.0
    to_str = f"-to {segments[i+1]['start']} " if i != len(segments)-1 else ""
    cmd = f"{ffmpeg_path} -loglevel 8 -y " \
        + f"-accurate_seek -ss {segments[i]['start']} " \
        + to_str \
        + f"-i {vid_file} " \
        + f"-i cache/{i}.wav " \
        + f"-c:v {video_codec} " \
        + f"-c:a {audio_codec} -ab {audio_bitrate} " \
        + f"-filter_complex \"[0:v]setpts={pts_ratio:.2f}*PTS[v];[0:a][1]amix=inputs=2:duration=longest:weights='0.05 1'[a]\" " \
        + f"-map \"[v]\" -map \"[a]\" cache/{i}_video.mp4"
    os.system(cmd)

CPU times: user 14.2 ms, sys: 86 µs, total: 14.3 ms
Wall time: 14 s


## Combine resulted audio with youtube video

In [37]:
list = []
for i in range(0, len(segments)):
    list.append(f'file {i}_video.mp4')
    
with open("cache/list.txt", "w") as text_file:
    text_file.write("\n".join(list))

os.system(f'ffmpeg -loglevel 8 -y -f concat -i cache/list.txt -c copy result.mp4')

0